### 卷积神经网络进行图像识别

卷积神经网络的图像识别模仿了人脑的视觉处理机制，采用分级提取特征的原理，每一级的特征均由网络学习提取，识别效果优于人工选取特征的算法
例如：在人脸识别过程中，最底层特征基本是各个方向上的边缘，越往上的神经层越能提取出人脸的局部特征，最上层由不同的高级特征组合成人脸的图像

卷积：将过滤器每个格子的权重值与图片对应的像素值相乘并累加，得到的值就是特征图中的一个像素值
    相比传统全连接的神经网络，能有效减少权重数
    传统：100*100 图片每个像素点都连接到每一个隐含层的节点上，如果隐含层的节点数为 10000, 那么连接的权重总数为 10^8 个
    卷积: 10*10 过滤器对原图 100*100 图片 进行卷积时，该过滤器不断滑动对应生成一张特征图，即一个过滤器(100个权重值)可对应一张特征图，如果有 100 张特征图，则一共只需要 10^4 个权重值
    所以在一个隐含层的情况下，卷积神经网络的权重数目可以减小至全连接神经网络权重数目的万分之一，大大减少计算量，提高计算效率
实际训练中，第一层的每一个过滤器的权重值会不断地被更新优化，使得每个过滤器的可视化纹理模式基本上反映了各个方向上的边缘特征

池化：降低数据的维度，过程就是 下采样，　特征图 8*8, 池化窗口 4*4, 采样结束后成 2*2 的池化特征图

In [1]:
import os
import torch
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as Data
from torchvision import datasets, transforms

In [2]:
# 图像数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,),)
])

In [3]:
train_set = datasets.MNIST('data', train=True, download=True, transform=transform)
test_set = datasets.MNIST('data', train=False, download=True, transform=transform)

In [4]:
class LeNet(nn.Module):
    '''
    2d: 2维图片处理
    '''
    def __init__(self):
        super(LeNet, self).__init__()
        # c1卷积层 (1, 6, 5)输入 1 张灰度图, 输出 6 张特征图, 过滤器 5*5
        self.c1 = nn.Conv2d(1, 6, 5)
        # c3卷积层 (6, 16, 5)输入 6 张灰度图, 输出 16 张特征图, 过滤器 5*5
        self.c3 = nn.Conv2d(6, 16, 5)
        # fc1全连接层 池化层 S4 中的所有特征点 16*4*4, 全连接到 120 个点
        self.fc1 = nn.Linear(16*4*4, 120)
        # fc2全连接层 120 全连接到 84 个点
        self.fc2 = nn.Linear(120, 84)
        # fc3全连接层 84 全连接到 10 个输出
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # max_pool2d 池化, 池化核 2*2
        x = F.max_pool2d(F.relu(self.c1(x)), 2)
        x = F.max_pool2d(F.relu(self.c3(x)), 2)
        # 转化一维向量
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
        
    def num_flat_features(self, x):
        # 计算 x 特征点的总数
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [6]:
model = LeNet()
# 使用 DataLoader 来加载数据, batch_size 表示一次加载的数量, shuffle遍历不同批次的数据打乱顺序,num_workers 使用 2 个子进程加载数据
trainloader = torch.utils.data.DataLoader(train_set, batch_size=4, shuffle=True, num_workers=2)

In [7]:
def train(trainloader, model, epochs=1, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    # 带动量的随机梯度下降法
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader):
            inputs, labels = data
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 1000 == 999:
                print('[Epoch:%d, Batch:%5d] Loss: %.3f' % (epoch+1, i+1, running_loss/1000))
                running_loss = 0.0
    
    print('Finished Training')

In [8]:
train(trainloader, model, epochs=2)

[Epoch:1, Batch: 1000] Loss: 1.116
[Epoch:1, Batch: 2000] Loss: 0.281
[Epoch:1, Batch: 3000] Loss: 0.217
[Epoch:1, Batch: 4000] Loss: 0.170
[Epoch:1, Batch: 5000] Loss: 0.127
[Epoch:1, Batch: 6000] Loss: 0.115
[Epoch:1, Batch: 7000] Loss: 0.112
[Epoch:1, Batch: 8000] Loss: 0.100
[Epoch:1, Batch: 9000] Loss: 0.098
[Epoch:1, Batch:10000] Loss: 0.109
[Epoch:1, Batch:11000] Loss: 0.102
[Epoch:1, Batch:12000] Loss: 0.092
[Epoch:1, Batch:13000] Loss: 0.090
[Epoch:1, Batch:14000] Loss: 0.079
[Epoch:1, Batch:15000] Loss: 0.076
[Epoch:2, Batch: 1000] Loss: 0.067
[Epoch:2, Batch: 2000] Loss: 0.056
[Epoch:2, Batch: 3000] Loss: 0.061
[Epoch:2, Batch: 4000] Loss: 0.072
[Epoch:2, Batch: 5000] Loss: 0.066
[Epoch:2, Batch: 6000] Loss: 0.064
[Epoch:2, Batch: 7000] Loss: 0.056
[Epoch:2, Batch: 8000] Loss: 0.068
[Epoch:2, Batch: 9000] Loss: 0.054
[Epoch:2, Batch:10000] Loss: 0.060
[Epoch:2, Batch:11000] Loss: 0.043
[Epoch:2, Batch:12000] Loss: 0.037
[Epoch:2, Batch:13000] Loss: 0.059
[Epoch:2, Batch:1400

In [ ]:
def load_param(model, path):
    if os.path.exists(path):
        model.load_state_dict(torch.load(path))

def save_param(model, path):
    torch.save(model.state_dict(), path)

In [11]:
model.parameters

<bound method Module.parameters of LeNet(
  (c1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (c3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

In [12]:
testloader = torch.utils.data.DataLoader(test_set, batch_size=4, shuffle=True, num_workers=2)

In [13]:
def test(testloader, model):
    # 正确的总数
    correct = 0
    # 预测的总数
    total = 0
    for data in testloader:
        image, labels = data
        outputs = model(image)
        _, perdicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (perdicted == labels).sum()
    print('Accuracy on the test set: %d %%' % (100 * correct / total))

In [14]:
test(testloader, model)

Accuracy on the test set: 98 %


In [ ]:
torch.save(model, 'data/model_num_image.pkl')